# Backend – Dungeon Mice

## Estado inicial
Se parte de un script en Python con OpenCV que:
- Lee un video del experimento
- Detecta movimiento
- Dibuja regiones y contornos

Este código funciona visualmente, pero:
- Mezcla visión, lógica y visualización
- No mantiene estado del ratón
- No mide tiempos ni eventos
- No es reutilizable por el equipo de GUI

---

## Objetivo
Desarrollar un **backend limpio** que:
- Detecte la posición del ratón
- Determine la región en la que se encuentra
- Detecte entradas y salidas de regiones
- Calcule el tiempo acumulado en cada región
- Exporte datos claros para que la GUI los consuma

El backend **no maneja interfaz gráfica**.

---

## Por qué se hace así
Separar backend y GUI permite:
- Probar la lógica sin interfaz
- Detectar errores antes de integrar
- Mantener datos confiables del experimento
- Evitar dependencias innecesarias entre equipos

---

## Plan de acción
1. Separar visión, lógica y visualización  
2. Definir el estado del ratón y las métricas  
3. Modelar las regiones como objetos  
4. Probar la lógica sin video (datos simulados)  
5. Validar el backend usando el video del experimento  

---

## Validación
El backend debe:
- Funcionar sin GUI
- Poder probarse con datos falsos y con video
- Permitir verificar resultados por consola o logs

---

## Relación con GUI
- La GUI importa nuestras funciones
- La GUI se encarga de la interfaz
- El backend garantiza que los datos sean correctos

---

## Resultado esperado
- Backend estable y testeado
- Lógica clara y defendible
- Base sólida para integración con GUI


backend/
│
├── regions.py        # Geometría y definición de zonas
├── tracker.py        # Detección del ratón (OpenCV)
├── logic.py          # Entradas, salidas, tiempos
├── experiment.py     # Unir todo


In [1]:
import cv2
import numpy as np

In [4]:
cap = cv2.VideoCapture("Mice maze experiment.mp4")

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))

while True:
	ret, frame = cap.read()
	if ret == False: break

	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	color = (0,255,0)
	# Areas importantes

	#norte = np.array([[567,354],[570,95],[615,100],[618,354]])
	#sur = np.array([[545,705],[566,417],[616,412],[610,707]])
	este = np.array([[620,450],[903,450],[900,320],[622,320]])
	oeste = np.array([[272,450],[274,320],[566,320],[562,450]])

	# imagenes auxiliares
	imAux = np.zeros(shape = frame.shape[:2],dtype= np.uint8)
	#imAux_norte = cv2.drawContours(imAux, [norte], -1, (255), -1)

	#imAux_sur = cv2.drawContours(imAux, [sur], -1, (255), -1)

	imAux_este = cv2.drawContours(imAux, [este], -1, (255), -1)


	imAux_norte = cv2.drawContours(imAux, [oeste], -1, (255), -1)

	areas_img = cv2.bitwise_and(gray, gray, mask = imAux)

	# Substraccion de fondo

	fgmask = fgbg.apply(areas_img)
	fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
	fgmask = cv2.dilate(fgmask, None, iterations = 2)

	# Filtrar por area de contorno
	cnts = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
	for cnt in cnts:
		if cv2.contourArea(cnt) > 2000:
			x,y,w,h = cv2.boundingRect(cnt)
			cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255),2)

			

	#cv2.drawContours(frame, [norte], -1, color,2)
	#cv2.drawContours(frame, [sur], -1, color,2)
	cv2.drawContours(frame, [este], -1, color,2)
	cv2.drawContours(frame, [oeste], -1, color,2)

	cv2.imshow("frame", frame)
	cv2.imshow("fgmask", fgmask)

	k = cv2.waitKey(30) & 0xFF
	if k == 27:
		break

cap.release
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [3]:
# Prueba de automatización

cap = cv2.VideoCapture("mice_dungeon.mp4")

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))



# Función que maneja el evento del mouse
def select_points(event, x, y, flags, param):
    points = np.zeros(shape = (4,2))
    
    if event == cv2.EVENT_LBUTTONDOWN:  # Si se hace clic izquierdo
        # Guardar el punto seleccionado

        if len(points) == 4:  # Si ya se seleccionaron dos puntos
            cv2.rectangle(image, points[0], points[1], (0, 255, 0), 2)  # Dibujar rectángulo
            cv2.imshow("Imagen", image)

[ WARN:0@0.185] global cap_gstreamer.cpp:1436 open OpenCV | GStreamer warning: Error opening bin: unexpected reference "mice_dungeon" - ignoring
[ WARN:0@0.185] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
